In [19]:
import pandas as pd
import os
import sys 
import numpy as np
from functools import reduce
import itertools
import  combined_country_regions 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [20]:

desired_width=1000
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns',10)
pd.options.display.max_rows = 50

class baci:
    def readindata(self, bacidata, verbose = False, tmp_save = True) -> pd.DataFrame:
        df1 = pd.read_csv(bacidata, usecols=['t','i','j','k','v','q'], 
                          dtype= {'t': 'int64',
                                  'i': 'int64', 
                                  'j': 'int64', 
                                  'k': 'object',
                                  'v': 'float64',
                                  'q': 'object'}
                          )

        # This is too complicated, but '   NA' should be converted to float
        df1['q'] = df1['q'].apply(lambda x: x.strip())
        df1['q'].replace('NA', np.nan, inplace=True)
        df1['q'] = df1['q'].astype(float)

        #dimensions
        #print("shape", df1.shape)

        # rename columns to make them meaningful
        df1.rename(columns={'t': 'Year', 'i': 'Exporter', 'j': 'Importer', 'k': 'Product', 'v': 'Value', 'q': 'Quantity'}, inplace=True)

        # replace number with name of country exporter
        iso1 = pd.read_csv(r"C:\Users\jpark\Downloads\BACI_HS92_V202401b\country_codes_V202401b.csv", usecols=['country_code', 'country_iso3'])
        df1 = df1.merge(iso1, left_on="Exporter", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Exporter'], inplace = True)
        df1.rename(columns={"country_iso3": "Exporter"}, inplace=True)
    
        # replace number with name of country importer
        df1 = df1.merge(iso1, left_on="Importer", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Importer'], inplace = True)
        df1.rename(columns={"country_iso3": "Importer"}, inplace=True)

        if verbose:
            hcodes = [str(x)[0:2] for x in df1["Product"]]
            print(set(hcodes))
            print(len(set(hcodes)))

        return df1
    
    def addprodcode(self, data):
        # add product_codes
        prodcodes = pd.read_csv(r"C:\Users\jpark\Downloads\BACI_HS92_V202401b\product_codes_HS92_V202401b.csv", usecols=['code', 'description'])
        mask = prodcodes['code'] == '9999AA'
        prodcodes = prodcodes[~mask]
        #prodcodes['code'] = prodcodes['code'].astype('int64')
        data = data.merge(prodcodes, left_on = "Product", right_on = "code", how = "left")
        
        return data
       
    def subsetData(self, data_param: pd.DataFrame(), iso3_param: list[str], imp_exp_param: str, products_param: list[str], minvalue_param=0.0) -> pd.DataFrame():
        df1 = data_param.copy()
        if products_param:
            out1 = df1[(df1[imp_exp_param].isin(iso3_param)) & (df1["Product"].isin(products_param))]
            out1.sort_values(by=['Value', 'Importer'], inplace=True, ascending=False)
            out1 = out1[out1['Value'] >= minvalue_param]
            out1['Value'] = out1['Value'].astype(int)
        else: # return all products
            out1 = df1[df1[imp_exp_param].isin(iso3_param)]
            out1.sort_values(by=['Value', 'Importer'], inplace=True, ascending=False)
            out1 = out1[out1['Value'] >= minvalue_param]
            out1['Value'] = out1['Value'].astype(int)

        return out1
    
    def subsetStrategicGoods(self, data, strategicProducts: list):
        df1 = data.copy()
        df2 = df1[df1['Product'].isin(strategicProducts)]
        return df2
    
    def addregion(self, data, exim):
        if exim == "Exporter":
            iso_regions = pd.read_csv(r"src\baci\data\iso_countries_regions.csv")
            iso_regions = iso_regions[['alpha-3', 'region']]
            data = data.merge(iso_regions, left_on="Exporter", right_on="alpha-3", how="left")
            data.rename(columns = {'region': 'Exporter_Region'}, inplace = True)
            data.drop(columns=["alpha-3"], inplace=True)
        elif exim == "Importer":
            iso_regions = pd.read_csv(r"src\baci\data\iso_countries_regions.csv")
            iso_regions = iso_regions[['alpha-3', 'region']]
            data = data.merge(iso_regions, left_on="Importer", right_on="alpha-3", how="left")
            data.rename(columns = {'region': 'Importer_Region'}, inplace = True)
            data.drop(columns=["alpha-3"], inplace=True)
        else: 
            print("Error")

        return data

    def addshortdescriptoProdname(self, data):
        localdata = data.copy()
        prod_h6 = pd.read_csv(r"src\baci\data\hs6twodigits.csv", dtype = str)

        # this is necessary because codes 1:9 should be 01:09
        localdata.loc[:, 'code'] = ["0" + x if len(x) == 5 else x for x in localdata['Product'].astype(str)]

        localdata['shrtDescription'] = localdata['code'].astype(str).str[0:2]
        proddesc = localdata.merge(prod_h6, left_on="shrtDescription", right_on="code")
        proddesc.drop(columns = {'code_x', 'shrtDescription', 'code_y'}, inplace = True)

        proddesc.rename(columns = {"product": "code"}, inplace = True)

        return proddesc
    
    def addlongdescription(self, data):
        localdata = data.copy()
        longdesc = pd.read_csv(r"C:\Users\jpark\Downloads\BACI_HS92_V202401b\product_codes_HS92_V202401b.csv", dtype = str)
        longdesc.rename(columns = {"code": "isocode"}, inplace=True)
        longproddesc = localdata.merge(longdesc, left_on="Product", right_on="isocode", how = 'left', suffixes = ['x', 'y'])
       
        r1 = localdata.shape[0]
        r2 = longproddesc.shape[0]
        assert r1 == r2

        return longproddesc
        
    def valueacrossallcountries(self, data_param: pd.DataFrame()):
        ### Relative size of Step1 inputs per product
        g = data_param[['Product', 'Value']].groupby(['Product']).sum()
        valueofStep1products = g.apply(lambda x: x.sort_values(ascending=False))
        valueofStep1products['Percentage'] = 100 * (valueofStep1products / valueofStep1products['Value'].sum())

        print(valueofStep1products)

        return valueofStep1products

    def valuepercountryacrossprods(self, data_param, imp_exp_param):
        ### Relative size of Step1 inputs per exporter
        g = data_param[[imp_exp_param, 'Value']].groupby([imp_exp_param]).sum()
        valueofStep1perExporter = g.apply(lambda x: x.sort_values(ascending=False))
        valueofStep1perExporter['Percentage'] = 100 * (valueofStep1perExporter / valueofStep1perExporter['Value'].sum())

        print(valueofStep1perExporter)
        return valueofStep1perExporter

    def valueperprod(self, data_param, imp_exp_param):

        exp1 = data_param[['Value', imp_exp_param, 'Product']]
        g = exp1.groupby([imp_exp_param, 'Product']).sum().reset_index()  #this is now a data frame

        allprods = []
        for p in g['Product'].unique():
            prod = g[g['Product'] == p]
            prod.sort_values(by = ['Value'], ascending=False, inplace=True)
            allprods.append(prod)

        print(pd.concat(allprods))

        return pd.concat(allprods)

    def OECD_agg(self, data_param, baci_countries_param, imp_exp_param):
        print(imp_exp_param)

        assert (imp_exp_param == 'Exporter_ISO3') or (imp_exp_param == 'Importer_ISO3'), "needs to be Exporter_ISO3 or Importer_ISO3"

        grp_perCountry = data_param[['Value', imp_exp_param]].groupby([imp_exp_param]).sum().reset_index()
        merged1 = grp_perCountry.merge(baci_countries_param[['ISO3', 'OECD']], left_on=imp_exp_param, right_on="ISO3",
                                   how="left")

        out = merged1[[imp_exp_param, 'Value', 'OECD']].groupby(['OECD']).sum().reset_index()
        out['Percentage'] = 100 * (out['Value'] / out['Value'].sum())
        out.sort_values(['Percentage'], ascending=False,inplace=True)
        print(out)

        return out

# ININTIALIZE object
bc1 = baci()


In [21]:
def aluminium_thoughtime():
    yearsData = np.arange(1995, 2023, step=1)
    yearly = []
    for i in yearsData:
        print(i)
        bacidata = r"C:\\Users\\jpark\\Downloads\\BACI_HS92_V202401b\BACI_HS92_Y" + str(i) + "_V202401b.csv"

        data = bc1.readindata(bacidata, verbose = False, tmp_save = False)
        data = bc1.addlongdescription(data)

        selectthese = [x for x in data['description'] if "Aluminium" in x or "aluminium" in x]
        
        alum = data[data['description'].isin(selectthese)]
        perstate = alum[['Value', 'Exporter']].groupby(['Exporter']).sum()
        perstate.rename(columns = {"Value": str(i)}, inplace = True)
        yearly.append(perstate)

    return yearly

alm =aluminium_thoughtime()

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [25]:
alm1 = pd.concat(alm, axis=1)
alm1.to_csv("exportsAluminium.csv", float_format='%.0f')
alm1

,1995,1996,1997,1998,1999,...,2018,2019,2020,2021,2022
Exporter,,,,,,,,,,,
ABW,181.305,74.028,79.122,140.313,256.509,...,936.004,1135.070,693.888,1438.847,1608.701
AFG,29.929,48.107,397.276,488.463,159.826,...,534.540,1732.443,1280.023,4197.027,3495.550
AGO,1417.015,303.178,193.512,467.752,273.598,...,23602.235,19885.113,15527.189,28055.931,23629.599
AIA,0.949,6.070,3.713,NaN,NaN,...,29.180,61.492,1.000,0.198,13.907
ALB,3321.361,4121.709,4676.847,4648.782,4625.916,...,60599.340,48308.257,54616.985,81664.160,134006.472
...,...,...,...,...,...,...,...,...,...,...,...
CUW,NaN,NaN,NaN,NaN,NaN,...,2210.910,2594.756,1203.642,2409.610,3037.381
SXM,NaN,NaN,NaN,NaN,NaN,...,925.274,714.449,403.818,544.039,426.378
SSD,NaN,NaN,NaN,NaN,NaN,...,NaN,0.117,0.171,45.308,77.763
